In [ ]:
import cv2
import numpy as np
import threading
import pyfirmata
import time
import socket
import struct
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from filterpy.kalman import KalmanFilter

# --- Cài đặt camera ---
ws, hs = 1280, 720
cap = cv2.VideoCapture(0)
cap.set(3, ws)
cap.set(4, hs)

# --- YOLO + DeepSort ---
model = YOLO("model.pt")
tracker = DeepSort(
    max_age=150,              # Tăng thời gian giữ track khi mất tín hiệu
    max_iou_distance=0.3,    # Tăng yêu cầu gần giống hơn để tránh gộp sai
    nn_budget=100,
    embedder='mobilenet',
    n_init=5                 # Cần nhiều frame hơn để xác nhận một track
)


# --- Arduino ---
board = pyfirmata.Arduino("COM10")
servo_pinX = board.get_pin('d:9:s')
servo_pinY = board.get_pin('d:10:s')

# --- Kalman Filter ---
kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 1, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 1, 1],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 0, 1, 0]])
kf.P *= 1000
kf.x = np.array([0, 0, 0, 0])

# --- Socket Server ---
HOST, PORT = '192.168.11.208', 65432
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(1)
print("📡 Đang chờ Master kết nối...")
conn, addr = server_socket.accept()
print(f"🔗 Kết nối từ {addr}")

# --- Shared ---
latest_frame = None
detections = []
selected_feature = None
tracked_id = None
yolo_lock = threading.Lock()

# --- Servo control ---
current_servo_x, current_servo_y = 90, 90
threshold_error = 50
step = 1.5
servo_pinX.write(current_servo_x)
servo_pinY.write(current_servo_y)

# --- Hàm chuẩn hoá đặc trưng ---
def normalize_feature(f):
    norm = np.linalg.norm(f)
    return f / norm if norm != 0 else f

# --- Nhận dữ liệu đặc trưng từ Master ---
def receive_feature_loop():
    global selected_feature, tracked_id
    while True:
        try:
            raw_len = conn.recv(4)
            if not raw_len:
                continue
            msg_len = struct.unpack(">I", raw_len)[0]
            data = b''
            while len(data) < msg_len:
                packet = conn.recv(msg_len - len(data))
                if not packet:
                    break
                data += packet
            if data:
                selected_feature = normalize_feature(np.frombuffer(data, dtype=np.float32))
                tracked_id = None
                print(f"📥 Nhận đặc trưng: {selected_feature.shape}")
        except Exception as e:
            print(f"❌ Lỗi nhận đặc trưng: {e}")
            break

threading.Thread(target=receive_feature_loop, daemon=True).start()

# --- YOLO phát hiện ---
def yolo_thread():
    global detections, latest_frame
    while True:
        if latest_frame is not None:
            with yolo_lock:
                frame = latest_frame.copy()
            results = model(frame)
            dets = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())
                if cls == 0:
                    dets.append(([x1, y1, x2 - x1, y2 - y1], conf, cls))
            detections = dets

threading.Thread(target=yolo_thread, daemon=True).start()

# --- Main loop ---
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    with yolo_lock:
        latest_frame = frame.copy()

    tracks = tracker.update_tracks(detections, frame=frame)

    # --- Xử lý so khớp đặc trưng ---
    if selected_feature is not None:
        valid_tracks = [
            t for t in tracks
            if t.is_confirmed() and hasattr(t, "features") and t.features
        ]
        if valid_tracks:
            track_features = np.array([
                normalize_feature(t.features[-1]) for t in valid_tracks
            ])
            sims = track_features @ selected_feature
            best_idx = np.argmax(sims)
            best_score = sims[best_idx]

            if best_score > 0.78:
                best_match = valid_tracks[best_idx]
                if tracked_id != best_match.track_id:
                    tracked_id = best_match.track_id
                    print(f"🎯 Đã so khớp với ID: {tracked_id} | Score: {best_score:.4f}")
            else:
                print(f"⚠️ Không có đối tượng khớp mạnh (score={best_score:.4f}), giữ ID hiện tại: {tracked_id}")

    # --- Kiểm tra nếu tracked_id không còn trong danh sách hiện tại ---
    if tracked_id is not None and all(t.track_id != tracked_id for t in tracks):
        print(f"❌ Mất dấu ID {tracked_id}, reset...")
        tracked_id = None

    # --- Điều khiển servo ---
    for track in tracks:
        if track.track_id == tracked_id:
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            kf.predict()
            kf.update([cx, cy])
            px, py = kf.x[0], kf.x[2]
            err_x, err_y = (ws // 2) - px, (hs // 2) - py
            if abs(err_x) > threshold_error:
                current_servo_x += step if err_x > 0 else -step
            if abs(err_y) > threshold_error:
                current_servo_y += step if err_y > 0 else -step
            current_servo_x = np.clip(current_servo_x, 0, 180)
            current_servo_y = np.clip(current_servo_y, 0, 180)
            servo_pinX.write(current_servo_x)
            servo_pinY.write(current_servo_y)
            print(f"🎯 Servo hướng về ({int(px)}, {int(py)}) với ID {tracked_id}")
            break

    # --- Vẽ ---
    for track in tracks:
        if track.is_confirmed():
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            if track.track_id == tracked_id:
                color = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"ID {track.track_id}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Slave", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
conn.close()
server_socket.close()


📡 Đang chờ Master kết nối...
🔗 Kết nối từ ('192.168.11.93', 60092)

0: 384x640 1 toothbrush, 188.1ms
Speed: 12.0ms preprocess, 188.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 toothbrush, 162.4ms
Speed: 5.8ms preprocess, 162.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 toothbrushs, 134.7ms
Speed: 3.1ms preprocess, 134.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 toothbrush, 164.4ms
Speed: 2.5ms preprocess, 164.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 99.5ms
Speed: 2.3ms preprocess, 99.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 102.0ms
Speed: 1.6ms preprocess, 102.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 121.0ms
Speed: 1.4ms preprocess, 121.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 

In [2]:
import cv2
import urllib.request
import numpy as np

url = 'http://192.168.1.226:81/stream'
stream = urllib.request.urlopen(url)

bytes_buffer = b''

while True:
    bytes_buffer += stream.read(1024)
    a = bytes_buffer.find(b'\xff\xd8')  # JPEG start
    b = bytes_buffer.find(b'\xff\xd9')  # JPEG end

    if a != -1 and b != -1:
        jpg = bytes_buffer[a:b+2]
        bytes_buffer = bytes_buffer[b+2:]

        img = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)

        # Nếu cần xoay chiều ảnh
        img = cv2.rotate(img, cv2.ROTATE_180)

        cv2.imshow('ESP32 Stream', img)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import threading
import pyfirmata
import time
import urllib.request
import socket
import struct
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from filterpy.kalman import KalmanFilter
from http.client import IncompleteRead

# --- Cài đặt stream từ ESP32 --- 
url = 'http://192.168.1.226:81/stream'
stream = urllib.request.urlopen(url)
bytes_buffer = b''

# --- Thông số ảnh ---
ws, hs = 1280, 720  # bạn có thể thay đổi nếu cần

# --- YOLO + DeepSort ---
model = YOLO("model.pt")
tracker = DeepSort(
    max_age=30,
    max_iou_distance=0.3,
    nn_budget=100,
    embedder='mobilenet',
    n_init=5
)

# --- Arduino ---
board = pyfirmata.Arduino("COM3")
servo_pinX = board.get_pin('d:9:s')
servo_pinY = board.get_pin('d:10:s')

# --- Kalman Filter ---
kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 1, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 1, 1],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 0, 1, 0]])
kf.P *= 1000
kf.x = np.array([0, 0, 0, 0])

# --- Socket Server ---
HOST, PORT = '192.168.1.241', 65432
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(1)
print("📡 Đang chờ Master kết nối...")
conn, addr = server_socket.accept()
print(f"🔗 Kết nối từ {addr}")

# --- Shared ---
latest_frame = None
detections = []
selected_feature = None
tracked_id = None
yolo_lock = threading.Lock()

# --- Servo control ---
current_servo_x, current_servo_y = 90, 90
threshold_error = 7
step = 1.5
servo_pinX.write(current_servo_x)
servo_pinY.write(current_servo_y)

# --- Hàm chuẩn hoá đặc trưng ---
def normalize_feature(f):
    norm = np.linalg.norm(f)
    return f / norm if norm != 0 else f

# --- Nhận dữ liệu đặc trưng từ Master ---
def receive_feature_loop():
    global selected_feature, tracked_id
    while True:
        try:
            raw_len = conn.recv(4)
            if not raw_len:
                continue
            msg_len = struct.unpack(">I", raw_len)[0]
            data = b''
            while len(data) < msg_len:
                packet = conn.recv(msg_len - len(data))
                if not packet:
                    break
                data += packet
            if data:
                selected_feature = normalize_feature(np.frombuffer(data, dtype=np.float32))
                tracked_id = None
                print(f"📥 Nhận đặc trưng: {selected_feature.shape}")
        except Exception as e:
            print(f"❌ Lỗi nhận đặc trưng: {e}")
            break

threading.Thread(target=receive_feature_loop, daemon=True).start()

# --- YOLO phát hiện ---
def yolo_thread():
    global detections, latest_frame
    while True:
        if latest_frame is not None:
            with yolo_lock:
                frame = latest_frame.copy()
            results = model(frame)
            dets = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())
                if cls == 0:
                    dets.append(([x1, y1, x2 - x1, y2 - y1], conf, cls))
            detections = dets

threading.Thread(target=yolo_thread, daemon=True).start()

def open_stream():
    return urllib.request.urlopen(url)

# --- Hàm lấy frame từ stream MJPEG ---
def get_frame_from_stream():
    global bytes_buffer, stream
    while True:
        try:
            bytes_buffer += stream.read(1024)
        except IncompleteRead as e:
            # Bắt lỗi đọc dữ liệu không đầy đủ, lấy phần đọc được
            bytes_buffer += e.partial
            # Có thể reset lại stream để tái kết nối
            stream.close()
            stream = open_stream()
            continue
        except Exception as e:
            print(f"Lỗi đọc stream: {e}")
            stream.close()
            stream = open_stream()
            continue

        a = bytes_buffer.find(b'\xff\xd8')  # JPEG start
        b = bytes_buffer.find(b'\xff\xd9')  # JPEG end

        if a != -1 and b != -1:
            jpg = bytes_buffer[a:b+2]
            bytes_buffer = bytes_buffer[b+2:]
            img = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            if img is not None:
                # Xoay chiều nếu cần
                img = cv2.rotate(img, cv2.ROTATE_180)
                return img
        # Nếu chưa tìm thấy frame hoàn chỉnh thì tiếp tục vòng lặp

# --- Main loop ---
while True:
    frame = get_frame_from_stream()
    if frame is None:
        print("⚠️ Không lấy được frame, thử lại...")
        continue

    with yolo_lock:
        latest_frame = frame.copy()

    tracks = tracker.update_tracks(detections, frame=frame)

    # --- Xử lý so khớp đặc trưng ---
    if selected_feature is not None:
        valid_tracks = [
            t for t in tracks
            if t.is_confirmed() and hasattr(t, "features") and t.features
        ]
        if valid_tracks:
            track_features = np.array([
                normalize_feature(t.features[-1]) for t in valid_tracks
            ])
            sims = track_features @ selected_feature
            best_idx = np.argmax(sims)
            best_score = sims[best_idx]

            if best_score > 0.75:
                best_match = valid_tracks[best_idx]
                if tracked_id != best_match.track_id:
                    tracked_id = best_match.track_id
                    print(f"🎯 Đã so khớp với ID: {tracked_id} | Score: {best_score:.4f}")
            else:
                print(f"⚠️ Không có đối tượng khớp mạnh (score={best_score:.4f}), giữ ID hiện tại: {tracked_id}")

    # --- Kiểm tra nếu tracked_id không còn trong danh sách hiện tại ---
    if tracked_id is not None and all(t.track_id != tracked_id for t in tracks):
        print(f"❌ Mất dấu ID {tracked_id}, reset...")
        tracked_id = None

    # --- Điều khiển servo ---
    for track in tracks:
        if track.track_id == tracked_id:
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            kf.predict()
            kf.update([cx, cy])
            px, py = kf.x[0], kf.x[2]
            err_x, err_y = (ws // 2) - px, (hs // 2) - py
            if abs(err_x) > threshold_error:
                current_servo_x += step if err_x > 0 else -step
            if abs(err_y) > threshold_error:
                current_servo_y += step if err_y > 0 else -step
            current_servo_x = np.clip(current_servo_x, 180, 0)
            current_servo_y = np.clip(current_servo_y, 180, 0)
            servo_pinX.write(current_servo_x)
            servo_pinY.write(current_servo_y)
            print(f"🎯 Servo hướng về ({int(px)}, {int(py)}) với ID {tracked_id}")
            break

    # --- Vẽ ---
    for track in tracks:
        if track.is_confirmed():
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            if track.track_id == tracked_id:
                color = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"ID {track.track_id}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Slave", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

conn.close()
server_socket.close()
